In [2]:
import os
import shutil
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
from torchvision.utils import save_image

# Dataset path
data_path = "/home/workstation/code/XAImethods/hf_cam_dev/ImageNet-Mini/images"

# Check if a file is a valid image file
def is_valid_image_file(filename):
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    return any(filename.lower().endswith(ext) for ext in valid_extensions)

# Check if a directory contains at least one valid image file
def is_valid_directory(directory):
    for filename in os.listdir(directory):
        if is_valid_image_file(filename):
            return True
    return False

# List all directories
all_directories = [d for d in os.listdir(data_path)]

# List valid directories
valid_directories = [d for d in all_directories if is_valid_directory(os.path.join(data_path, d))]

# Backup and remove invalid directories temporarily
backup_path = "/tmp/invalid_dirs_backup"
os.makedirs(backup_path, exist_ok=True)
for d in set(all_directories) - set(valid_directories):
    shutil.move(os.path.join(data_path, d), backup_path)

# Image preprocessing transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Now load the dataset
dataset = ImageFolder(root=data_path, transform=transform, is_valid_file=is_valid_image_file)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Restore the directories (optional based on whether you want to keep the structure intact)
for d in os.listdir(backup_path):
    shutil.move(os.path.join(backup_path, d), data_path)

# Load the model
model = resnet50(pretrained=True)
model.eval()
if torch.cuda.is_available():
    model = model.cuda()

# Define loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Adversarial Attack
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 224, 224),
    nb_classes=len(valid_directories)
)
attack = ProjectedGradientDescent(classifier, eps=0.03, max_iter=100, eps_step=0.01)

adversarial_samples = []
for inputs, labels in dataloader:
    if torch.cuda.is_available():
        inputs, labels = inputs.cuda(), labels.cuda()
    adversarial = attack.generate(x=inputs.cpu().numpy(), y=labels.cpu().numpy())
    adversarial_samples.extend(adversarial)

# Save Adversarial Samples
def save_adversarial_samples(adversarial_samples, original_filepaths, save_directory):
    for adv_sample, original_path in zip(adversarial_samples, original_filepaths):
        sub_dir, filename = os.path.split(original_path)
        _, sub_dir = os.path.split(sub_dir)
        save_sub_dir = os.path.join(save_directory, sub_dir)
        os.makedirs(save_sub_dir, exist_ok=True)
        save_path = os.path.join(save_sub_dir, filename)
        save_image(torch.tensor(adv_sample).permute(1, 2, 0), save_path)

save_directory = "/home/workstation/code/XAImethods/hf_cam_dev/attacked/ImageNet-Mini/images"
save_adversarial_samples(adversarial_samples, [sample[0] for sample in dataset.samples], save_directory)

print("Adversarial sample generation and saving completed!")


/home/workstation/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/workstation/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


KeyboardInterrupt: 

In [3]:
import os
import shutil
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from art.attacks.evasion import ProjectedGradientDescent
from art.estimators.classification import PyTorchClassifier
from torchvision.utils import save_image
import numpy as np

# Dataset path
data_path = "/home/workstation/code/XAImethods/hf_cam_dev/ImageNet-Mini/images"

# Check if a file is a valid image file
def is_valid_image_file(filename):
    valid_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    return any(filename.lower().endswith(ext) for ext in valid_extensions)

# Check if a directory contains at least one valid image file
def is_valid_directory(directory):
    for filename in os.listdir(directory):
        if is_valid_image_file(filename):
            return True
    return False

# List all directories
all_directories = [d for d in os.listdir(data_path)]

# List valid directories
valid_directories = [d for d in all_directories if is_valid_directory(os.path.join(data_path, d))]

# Backup and remove invalid directories temporarily
backup_path = "/tmp/invalid_dirs_backup"
os.makedirs(backup_path, exist_ok=True)
for d in set(all_directories) - set(valid_directories):
    shutil.move(os.path.join(data_path, d), backup_path)

# Image preprocessing transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# Now load the dataset
dataset = ImageFolder(root=data_path, transform=transform, is_valid_file=is_valid_image_file)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Restore the directories (optional based on whether you want to keep the structure intact)
for d in os.listdir(backup_path):
    shutil.move(os.path.join(backup_path, d), data_path)

# Load the model
model = resnet50(pretrained=True)
model.eval()
if torch.cuda.is_available():
    model = model.cuda()

# Define loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Adversarial Attack
classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 224, 224),
    nb_classes=len(valid_directories)
)
attack = ProjectedGradientDescent(classifier, eps=0.03, max_iter=100, eps_step=0.01)
# Save Adversarial Samples function
def save_adversarial_samples(adversarial_samples, original_filepaths, save_directory):
    for adv_sample, original_path in zip(adversarial_samples, original_filepaths):
        sub_dir, filename = os.path.split(original_path)
        _, sub_dir = os.path.split(sub_dir)
        save_sub_dir = os.path.join(save_directory, sub_dir)
        os.makedirs(save_sub_dir, exist_ok=True)
        save_path = os.path.join(save_sub_dir, filename)
        
        # Convert to tensor if it's a numpy array
        if isinstance(adv_sample, np.ndarray):
            adv_sample = torch.from_numpy(adv_sample)
        
        # Rescale tensor values to [0, 1]
        adv_tensor = (adv_sample - adv_sample.min()) / (adv_sample.max() - adv_sample.min())
        
        save_image(adv_tensor, save_path)


save_directory = "/home/workstation/code/XAImethods/hf_cam_dev/attacked/ImageNet-Mini/images"

# Generate and immediately save adversarial samples
for i, (inputs, labels) in enumerate(dataloader):
    if torch.cuda.is_available():
        inputs, labels = inputs.cuda(), labels.cuda()
    adversarial = attack.generate(x=inputs.cpu().numpy(), y=labels.cpu().numpy())
    
    # Ensure the adversarial samples are in the shape (batch_size, channels, height, width)
    assert adversarial.shape == (len(inputs), 3, 224, 224), f"Unexpected shape: {adversarial.shape}"
    
    # Corrected slicing operation to handle potential batch size mismatch
    start_idx = i * dataloader.batch_size
    end_idx = start_idx + len(inputs)
    save_adversarial_samples(adversarial, [sample[0] for sample in dataset.samples[start_idx:end_idx]], save_directory)
    
    print(f"Processed and saved batch {i+1} of {len(dataloader)}")

print("Adversarial sample generation and saving completed!")

/home/workstation/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/workstation/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processed and saved batch 1 of 122


Processed and saved batch 2 of 122


Processed and saved batch 3 of 122


Processed and saved batch 4 of 122


...

Processed and saved batch 118 of 122


Processed and saved batch 119 of 122


Processed and saved batch 120 of 122


Processed and saved batch 121 of 122


Processed and saved batch 122 of 122
Adversarial sample generation and saving completed!
